In [2]:
!pip install tabula-py pandas requests beautifulsoup4 openpyxl

  Using cached numpy-2.3.3-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
    --------------------------------------- 0.3/12.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.0 MB 1.7 MB/s eta 0:00:07
   --- ------------------------------------ 1.0/12.0 MB 2.1 MB/s eta 0:00:06
   ------ --------------------------------- 1.8/12.0 MB 2.4 MB/s eta 0:00:05
   ------- -------------------------------- 2.4/12.0 MB 2.5 MB/s eta 0:00:04
   ---------- ----------------------------- 3.1/12.0 MB 2.7 MB/s eta 0:00:04
   ------------- -------------------------- 3.9/12.0 MB 2.8 MB/s eta 0:00:03
   --------------- ------------------------ 4.7/12.0 MB 3.0 MB/s eta 0:00:03
   ------------------ --------------------- 5.5/12.0 MB 3.2 MB/s eta 0:00:03
   -------------------- -----------------

In [3]:
# Jupyter Notebook Cell 2
import requests
from bs4 import BeautifulSoup
import tabula
import pandas as pd
import os

# Create folders to store PDFs
os.makedirs("SPLG_fact_sheets", exist_ok=True)

# -----------------------------
# 1️⃣ Get list of SPLG fact sheets from SPDR site
# -----------------------------
url = "https://www.ssga.com/us/en/individual/etfs/funds/spdr-portfolio-s-p-500-etf-splg#literature"

headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Find all PDF links
pdf_links = []
for a in soup.find_all("a", href=True):
    href = a['href']
    if href.lower().endswith(".pdf") and "Fact Sheet" in a.text:
        pdf_links.append(href if href.startswith("http") else "https://www.ssga.com" + href)

print(f"Found {len(pdf_links)} fact sheets")


Found 0 fact sheets


In [4]:
# Jupyter Notebook Cell 3
# -----------------------------
# 2️⃣ Download PDFs
# -----------------------------
for link in pdf_links:
    filename = os.path.join("SPLG_fact_sheets", link.split("/")[-1])
    if not os.path.exists(filename):
        r = requests.get(link)
        with open(filename, "wb") as f:
            f.write(r.content)
print("Downloaded all PDFs")


Downloaded all PDFs


In [5]:
# Jupyter Notebook Cell 4
# -----------------------------
# 3️⃣ Extract holdings & sectors from PDFs
# -----------------------------
pdf_files = os.listdir("SPLG_fact_sheets")
all_holdings = []
all_sectors = []

for pdf in pdf_files:
    pdf_path = os.path.join("SPLG_fact_sheets", pdf)
    try:
        tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True)
        for table in tables:
            cols = [c.lower() for c in table.columns]
            
            # Top Holdings Table
            if 'ticker' in cols or 'symbol' in cols:
                table['Source_Date'] = pdf.split('_')[-1].replace('.pdf','')
                all_holdings.append(table)
            
            # Sector Allocation Table
            if 'sector' in cols or 'sector name' in cols:
                table['Source_Date'] = pdf.split('_')[-1].replace('.pdf','')
                all_sectors.append(table)
    except:
        print(f"Failed to read {pdf}")

# Combine dataframes
holdings_df = pd.concat(all_holdings, ignore_index=True) if all_holdings else pd.DataFrame()
sectors_df = pd.concat(all_sectors, ignore_index=True) if all_sectors else pd.DataFrame()

# Save to Excel
with pd.ExcelWriter('SPLG_holdings_sectors_5yr.xlsx') as writer:
    holdings_df.to_excel(writer, sheet_name='Top Holdings', index=False)
    sectors_df.to_excel(writer, sheet_name='Sector Allocation', index=False)

print("Saved combined holdings and sector data to SPLG_holdings_sectors_5yr.xlsx")


Saved combined holdings and sector data to SPLG_holdings_sectors_5yr.xlsx
